In [53]:
import pandas as pd
import numpy as np
week1 = pd.read_csv("Pool_results.csv")
#week1.head()


In [54]:
#replace NaN with 0
week1['Unnamed: 0'] = week1['Unnamed: 0'].replace(np.nan, 0)


In [55]:
#function to combine picks and point values
def combine(df1):
    my_file = df1
    rows = my_file.shape[0]
    i = 1 
    while i < rows:
        for col in my_file.columns:
            my_file[col][i+1] = int(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], week1[col][i+1]]
            #my_file.drop(i+1, axis=0, inplace=True)
        i = i+2
    return my_file


In [56]:
week1 = combine(week1)
#week1.drop(4, axis=0, inplace=True)
week1.head()

,Unnamed: 0,Game 1,Game 2,Game 3,Game 4,Game 5,Game 6,Game 7,Game 8,Game 9,Game 10,Game 11,Game 12,Game 13,Game 14,Game 15,Game 16
0,Winner,BUF,NO,BAL,PIT,MIA,WAS,CHI,CLE,TIE,PHI,LAC,MIN,NYG,KC,TB,SEA
1,"[abfarias, 0]","[BUF, 1]","[ATL, 16]","[NYJ, 15]","[CIN, 14]","[MIA, 13]","[WAS, 12]","[CHI, 11]","[CAR, 10]","[HOU, 9]","[DET, 8]","[LAC, 7]","[MIN, 6]","[TEN, 5]","[ARI, 4]","[DAL, 3]","[SEA, 2]"
2,0,1,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
3,"[Bob-Z, 0]","[BUF, 2]","[NO, 10]","[BAL, 14]","[CIN, 13]","[MIA, 6]","[WAS, 7]","[CHI, 1]","[CAR, 9]","[IND, 15]","[PHI, 8]","[LAC, 5]","[MIN, 4]","[TEN, 11]","[KC, 12]","[TB, 3]","[DEN, 16]"
4,0,2,10,14,13,6,7,1,9,15,8,5,4,11,12,3,16


In [3]:
#function to calculate total points
def total(picks, winner, points):
    total = 0
    for i in picks:
        if picks[i] == winner[i]:
            total = total + points[i]

    return total 